In [1]:
!pip install catboost seaborn openpyxl

Looking in indexes: http://mirror.ca.sbrf.ru/pypi/simple
     |████████████████████████████████| 76.1 MB 86.6 MB/s                            | 1.0 MB 86.6 MB/s eta 0:00:0101██▊                        | 18.4 MB 86.6 MB/s eta 0:00:01           | 27.0 MB 86.6 MB/s eta 0:00:01       | 36.1 MB 86.6 MB/s eta 0:00:01��██             | 45.1 MB 86.6 MB/s eta 0:00:01��█████▊         | 54.2 MB 86.6 MB/s eta 0:00:01███████████████████████▍     | 62.7 MB 86.6 MB/s eta 0:00:01███████████████████████▏ | 71.9 MB 86.6 MB/s eta 0:00:01
     |████████████████████████████████| 292 kB 86.9 MB/s            
     |████████████████████████████████| 242 kB 114.7 MB/s            
     |████████████████████████████████| 26.5 MB 90.5 MB/s            1�██████▎                 | 11.9 MB 90.5 MB/s eta 0:00:01��       | 20.3 MB 90.5 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 83.0 MB/s            


In [51]:
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm
import datetime as dt
from sklearn.preprocessing import OrdinalEncoder
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.utils.class_weight import compute_class_weight
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.set_option('max_column', None)
tqdm.pandas()

In [18]:
motiv = pd.concat([pd.read_csv(f'../../for_files/MSAP_Jan_{i}.csv',sep=';', names=['DATE_REPORT_SRC','DATE_MONTH_x','CLIENT_TID','EMPLOYEE_LOGIN','employee_position_gr',
                                                                                'TB_NAME','OSB_ID','VSP_ID','SOURCE_SYSTEM_ID','ID_VDMRB','ID_VDMRB_UNIQUE','PRODUCT_GROUP','PRODUCT_GROUP_ID','PRODUCT_SUBGROUP',
                                                                                'PRODUCT_SUBGROUP_ID','PRODUCT','PRODUCT_ID','OPERATION_ID','GROUP_CALC','PRODUCT_CALC','ID_CALC','QUANTITY_x','AMOUNT_RUR',
                                                                                'AMOUNT_CURR','CURRENCY','ACC','TERM','CLIENT_ID','AGREEMENT_NUM','CHANNEL','PRODUCT_SOURCE','IN_SUP','prod_kpi',
                                                                                'is_sbolpro','DATE_REPORT','DATE_REPORT_SRC_x','DATE_MONTH','EMPLOYEE_FIO','EMPLOYEE_ID','EMPLOYEE_POSITION',
                                                                                'EBIRTH','RBEGDA','RENDDA','TB_ID','urf_code_uni','SOURCE_SYSTEM','OPERATION','QUANTITY','CHANNEL_GR','IS_FRAUD',
                                                                                'DESCRIPTION','insert_date','IN_MOTIV','PRODUCT_NAME','vl_up','DATE_UPDATE',
                                                                                'PAYMENT_STATUS','epk_id','untb','fullname','birth','Табельный_клиента',
                                                                                'Отпуск',]) for i in ['1-10','11-13','14-18','19-21','22-26','27-31']])

In [19]:
motiv = motiv.drop(['DATE_REPORT_SRC_x', 'DATE_MONTH_x', 'QUANTITY_x'], axis=1)

In [5]:
motiv.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15577087 entries, 0 to 2750748
Data columns (total 60 columns):
 #   Column                Non-Null Count     Dtype  
---  ------                --------------     -----  
 0   DATE_REPORT_SRC       15577087 non-null  object 
 1   CLIENT_TID            10739595 non-null  float64
 2   EMPLOYEE_LOGIN        15577087 non-null  object 
 3   employee_position_gr  15577087 non-null  object 
 4   TB_NAME               15577087 non-null  object 
 5   OSB_ID                15577087 non-null  object 
 6   VSP_ID                15577087 non-null  object 
 7   SOURCE_SYSTEM_ID      15577087 non-null  object 
 8   ID_VDMRB              15577087 non-null  object 
 9   ID_VDMRB_UNIQUE       15577087 non-null  object 
 10  PRODUCT_GROUP         15577087 non-null  object 
 11  PRODUCT_GROUP_ID      15577076 non-null  float64
 12  PRODUCT_SUBGROUP      15577087 non-null  object 
 13  PRODUCT_SUBGROUP_ID   15577076 non-null  float64
 14  PRODUCT          

# Preprocessing

In [6]:
motiv.loc[(motiv['RENDDA'] > '2023-01-01'), 'RENDDA'] = '2262-1-1'
motiv = motiv.fillna({
    'EBIRTH': '1700-1-1',
    'RBEGDA': '1700-1-1',
    'RENDDA': '2262-1-1',
    'insert_date': '1700-1-1',
    'DATE_UPDATE': '1700-1-1',
    'EMPLOYEE_FIO':'-',
    'EMPLOYEE_POSITION': '-',
    'urf_code_uni': '-',
    'SOURCE_SYSTEM': '-',
    'OPERATION': '-',
    'QUANTITY': 1,
    'CHANNEL_GR': '-',
    'DESCRIPTION': '-',
    'PRODUCT_NAME': '-',
    'vl_up': 0,
    'fullname': '~'
})

In [7]:
motiv['DATE_REPORT'] = pd.to_datetime(motiv['DATE_REPORT']).dt.date
motiv['EBIRTH'] = pd.to_datetime(motiv['EBIRTH']).dt.date
motiv['RBEGDA'] = pd.to_datetime(motiv['RBEGDA']).dt.date
motiv['RENDDA'] = pd.to_datetime(motiv['RENDDA']).dt.date
motiv['insert_date'] = pd.to_datetime(motiv['insert_date']).dt.date
motiv['DATE_UPDATE'] = pd.to_datetime(motiv['DATE_UPDATE']).dt.date
motiv['birth'] = pd.to_datetime(motiv['birth']).dt.date
motiv['DATE_MONTH'] = pd.to_datetime(motiv['DATE_MONTH'])

In [20]:
motiv['ID_Operation_History'] = motiv.index

In [10]:
motiv = motiv.merge(motiv[['DATE_REPORT','EMPLOYEE_ID', 'ID_Operation_History']].groupby(['EMPLOYEE_ID','DATE_REPORT'], as_index=False).\
    agg({'ID_Operation_History':'count'}).rename(columns={'ID_Operation_History':'emp_oper_per_day'}), on=['DATE_REPORT','EMPLOYEE_ID'], how='left')
motiv = motiv.merge(motiv[['DATE_REPORT','epk_id', 'ID_Operation_History']].groupby(['epk_id','DATE_REPORT'], as_index=False).\
    agg({'ID_Operation_History':'count'}).rename(columns={'ID_Operation_History':'client_product_per_day'}), on=['DATE_REPORT','epk_id'], how='left')

In [12]:
time_series = motiv[['DATE_REPORT', 'ID_Operation_History', 'epk_id']]
time_series['DATE_REPORT'] = pd.to_datetime(motiv['DATE_REPORT'])
motiv['month'] = motiv['DATE_MONTH'].dt.month
motiv['year'] = motiv['DATE_MONTH'].dt.year

In [13]:
motiv = motiv.merge(motiv[['EMPLOYEE_ID', 'year', 'month', 'ID_Operation_History']].\
                groupby(['EMPLOYEE_ID', 'year', 'month'], as_index=False)[['ID_Operation_History']].count().\
          rename(columns={'ID_Operation_History':'emp_oper_per_month'}), on=['EMPLOYEE_ID', 'year', 'month'], how='left')

In [14]:
clprod = pd.read_csv('../for_files/Jan_client_product01_07-31_12.csv', sep=';')
clprod['DATE_REPORT'] = pd.to_datetime(clprod['DATE_REPORT'])

In [15]:
clprod = clprod.groupby(['DATE_REPORT','epk_id'], as_index=False).agg({'ID_Operation_History':'count'})

In [16]:
day_oper = time_series.groupby(['DATE_REPORT','epk_id'], as_index=False).agg({'ID_Operation_History':'count'})

In [17]:
day_oper['DATE_REPORT'][0] - dt.datetime(2021, 7, 1, 0, 0,0)

Timedelta('184 days 00:00:00')

In [18]:
dates = day_oper['DATE_REPORT'].drop_duplicates()

In [19]:
day_oper = pd.concat([clprod, day_oper])

In [20]:
dates.iloc[0] - dt.timedelta(181)

Timestamp('2021-07-04 00:00:00')

In [21]:
motiv['DATE_REPORT'] = pd.to_datetime(motiv['DATE_REPORT'])

In [22]:
oper_data = pd.DataFrame()
oper_per_data = pd.DataFrame()
for date in tqdm(dates):
    oper_per_data = day_oper[(day_oper['DATE_REPORT'] < date)&(day_oper['DATE_REPORT'] >= date - dt.timedelta(181))].\
                           groupby('epk_id', as_index=False).agg({'ID_Operation_History':'sum'})
    oper_per_data['DATE_REPORT'] = date
    oper_per_data = motiv[['ID_Operation_History','epk_id']][motiv['DATE_REPORT'] == date].merge(oper_per_data, on='epk_id', how='inner')
    oper_data = pd.concat([oper_data, oper_per_data])

  0%|          | 0/31 [00:00<?, ?it/s]

In [41]:
oper_data = oper_data[['epk_id', 'DATE_REPORT', 'client_product_at_moment']].drop_duplicates()

In [24]:
oper_data = oper_data.rename(columns={'ID_Operation_History_y': 'client_product_at_moment', 'ID_Operation_History_x': 'ID_Operation_History'})

In [13]:
norm_struct = motiv[['ID_Operation_History', 'DATE_REPORT', 'DATE_REPORT_SRC', 'DATE_MONTH', 'CLIENT_TID',
                     'EMPLOYEE_LOGIN', 'EMPLOYEE_FIO', 'EMPLOYEE_ID', 'EMPLOYEE_POSITION', 'employee_position_gr',
                     'TB_NAME', 'TB_ID', 'OSB_ID', 'VSP_ID', 'SOURCE_SYSTEM', 'SOURCE_SYSTEM_ID', 'ID_VDMRB',
                     'ID_VDMRB_UNIQUE', 'PRODUCT_GROUP', 'PRODUCT_GROUP_ID', 'PRODUCT_SUBGROUP', 'PRODUCT_SUBGROUP_ID',
                     'PRODUCT', 'PRODUCT_ID', 'OPERATION', 'OPERATION_ID', 'GROUP_CALC', 'PRODUCT_CALC', 'ID_CALC',
                     'QUANTITY', 'AMOUNT_RUR', 'CURRENCY', 'ACC', 'TERM', 'CLIENT_ID', 'AGREEMENT_NUM', 'CHANNEL',
                     'CHANNEL_GR', 'IS_FRAUD', 'IN_MOTIV', 'PRODUCT_SOURCE', 'PRODUCT_NAME', 'vl_up', 'PAYMENT_STATUS',
                     'IN_SUP', 'epk_id', 'prod_kpi', 'is_sbolpro']]

In [23]:
norm1, norm2, norm3, norm4 = np.array_split(norm_struct, 4)

In [27]:
norm4.to_csv('norm4.csv', sep=';')

In [30]:
motiv = motiv.merge(oper_data[['ID_Operation_History', 'client_product_at_moment']], on='ID_Operation_History', how='left')

In [38]:
motiv = motiv.drop('client_product_at_moment', axis=1)

In [ ]:
motiv = motiv.drop_duplicates()

In [ ]:
motiv['EBIRTH'] = np.where(motiv['EBIRTH'] > dt.date(1939,1,1), motiv['EBIRTH'], None)
motiv['RBEGDA'] = np.where(motiv['RBEGDA'] > dt.date(1939,1,1), motiv['RBEGDA'], None)
motiv['insert_date'] = np.where(motiv['insert_date'] > dt.date(1939,1,1), motiv['insert_date'], None)
motiv['DATE_UPDATE'] = np.where(motiv['DATE_UPDATE'] > dt.date(1939,1,1), motiv['DATE_UPDATE'], None)

In [ ]:
df = motiv[['ID_Operation_History', 'EMPLOYEE_POSITION', 'SOURCE_SYSTEM', 'OPERATION', 'CHANNEL_GR',
            'QUANTITY', 'PRODUCT_NAME', 'vl_up', 'emp_oper_per_day']]

In [ ]:
df['date_report'] = pd.to_datetime(motiv['DATE_REPORT']).dt.day
df['emp_age'] = (motiv['DATE_REPORT'] - motiv['EBIRTH']) / dt.timedelta(days=365)
df['report-start_job'] = (motiv['DATE_REPORT'] - motiv['RBEGDA']).dt.days
df['finish_job-report'] = (motiv['RENDDA'] - motiv['DATE_REPORT']).dt.days
df['insert-report'] = (motiv['insert_date'] - motiv['DATE_REPORT']).dt.days
df['update-report'] = (motiv['DATE_UPDATE'] - motiv['DATE_REPORT']).dt.days
df['client_age'] = (motiv['DATE_REPORT'] - motiv['birth']) / dt.timedelta(days=365)
df['client_is_emploee'] = np.where(motiv['Табельный_клиента'].isnull(), 0, 1)
df['emploee_in_vacation'] = np.where(motiv['Отпуск'].isnull(), 0, 1)
df['emp_oper_per_day'] = motiv['emp_oper_per_day']
df['emp_oper_per_month'] = motiv['emp_oper_per_month']
df['day_of_week'] = pd.to_datetime(df1['DATE_REPORT']).dt.day_name()
df['client_product_per_day'] = motiv['client_product_per_day']
df['target'] = np.where(motiv['IS_FRAUD'] != 0, 1, 0)

In [ ]:
df.to_csv('../nfs/')

In [ ]:
df.info(null_counts=True)

In [ ]:
mean_emp_age = df[df['emp_age'].notnull()]['emp_age'].mean()
mean_client_age = df[df['client_age'].notnull()]['client_age'].mean()

In [ ]:
data['emp_age'] = data['emp_age'].fillna(mean_emp_age)
data['client_age'] = data['client_age'].fillna(mean_client_age)

In [ ]:
data.loc[data['emp_age'] <= 0, ['emp_age']] = mean_emp_age
data['client_age'] = data['client_age'].fillna(mean_client_age)

# Catboost predict

In [5]:
data = pd.read_csv('../Jan_dataset_2.csv', index_col=0, sep=';', thousands=',')

In [6]:
data

,ID_Operation_History,EMPLOYEE_POSITION,SOURCE_SYSTEM,OPERATION,CHANNEL_GR,QUANTITY,PRODUCT_NAME,vl_up,emp_oper_per_day,date_report,emp_age,report-start_job,finish_job-report,insert-report,update-report,client_age,client_is_emploee,emploee_in_vacation,emp_oper_per_month,day_of_week,client_product_per_day,target,client_product_at_moment
0,0,?,АС Транзакт,Прием документов на получение кредита,?,100,Потребительские кредиты (кроме кредита под зал...,0,0.0,7,33.090103,-95947.0,87652,-58.0,38,34.723288,0,0,0.0,Friday,1.0,0,11.0
1,1,СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР,АС ФС,Оформление заявления на выдачу дебетовой банко...,ВСП,100,Дебетовая карта Сберкарта,4500,62.0,10,24.413699,1012.0,87649,-61.0,35,41.098630,0,0,777.0,Monday,4.0,0,1.0
2,2,СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР,АС ФС,Оформление заявления на выдачу банковских карт,ВСП,100,Личная дебетовая карта без овердрафта Сбербанк...,300,62.0,10,24.413699,1012.0,87649,-61.0,35,41.098630,0,0,777.0,Monday,4.0,0,1.0
3,3,СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР,АС Транзакт,Оформление заявления на выдачу банковских карт,ВСП,100,Кредитные карты (стандартные),12700,62.0,10,24.413699,1012.0,87649,-61.0,35,41.098630,0,0,777.0,Monday,4.0,0,1.0
4,4,СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР,АС ФС,Выдача банковской карты,ВСП,100,Личная дебетовая карта без овердрафта Сбербанк...,0,62.0,10,24.413699,1012.0,87649,-61.0,35,41.098630,0,0,777.0,Monday,4.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15577082,2750744,СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР,АС ФС,Выдача банковской карты,ВСП,100,Банковские карты,1400,48.0,31,26.663014,2128.0,87628,-82.0,14,42.058437,0,0,558.0,Monday,4.0,0,2.0
15577083,2750745,ВЕДУЩИЙ МЕНЕДЖЕР ПО ОБСЛУЖИВАНИЮ,АС ФС,Выдача банковской карты,ВСП,100,Банковские карты,1400,23.0,27,50.564384,1506.0,87632,-78.0,18,44.030137,0,0,310.0,Thursday,1.0,0,0.0
15577084,2750746,?,?,Установка МП Сбер ЕАптека из Личного кабинета ...,?,100,"Сервис ""Сбер ЕАптека""",0,0.0,30,33.090103,-95947.0,87629,-20.0,15,42.058437,0,0,0.0,Sunday,1.0,1,0.0
15577085,2750747,СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР,АС ЕРИБ,Оформление заявления на выдачу личной дебетово...,ВСП,100,Дебетовая карта Сберкарта,4500,30.0,31,47.115068,452.0,87628,-82.0,14,42.058437,0,0,507.0,Monday,4.0,0,2.0


In [7]:
model = CatBoostClassifier()

In [8]:
model.load_model('depth10_lr0_1')

In [9]:
cat_features = ['emploee_position', 'SOURCE_SYSTEM', 'OPERATION', 'CHANNEL_GR', 'PRODUCT_NAME', 'day_of_week']

In [10]:
X = data.drop(['target', 'ID_Operation_History'], axis=1).rename(columns={'EMPLOYEE_POSITION':'emploee_position'})

In [13]:
y = data['target']

In [14]:
test_dataset = Pool(X, y, cat_features=cat_features)

In [17]:
model.predict(test_dataset).sum()

1488911

In [50]:
print(classification_report(y, model.predict(test_dataset)))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96  13992172
           1       0.66      0.62      0.64   1584915

    accuracy                           0.93  15577087
   macro avg       0.81      0.79      0.80  15577087
weighted avg       0.93      0.93      0.93  15577087



In [22]:
motiv['predict'] = model.predict(test_dataset)

In [24]:
motiv[motiv['predict'] == 1]

,DATE_REPORT_SRC,CLIENT_TID,EMPLOYEE_LOGIN,employee_position_gr,TB_NAME,OSB_ID,VSP_ID,SOURCE_SYSTEM_ID,ID_VDMRB,ID_VDMRB_UNIQUE,PRODUCT_GROUP,PRODUCT_GROUP_ID,PRODUCT_SUBGROUP,PRODUCT_SUBGROUP_ID,PRODUCT,PRODUCT_ID,OPERATION_ID,GROUP_CALC,PRODUCT_CALC,ID_CALC,AMOUNT_RUR,AMOUNT_CURR,CURRENCY,ACC,TERM,CLIENT_ID,AGREEMENT_NUM,CHANNEL,PRODUCT_SOURCE,IN_SUP,prod_kpi,is_sbolpro,DATE_REPORT,DATE_MONTH,EMPLOYEE_FIO,EMPLOYEE_ID,EMPLOYEE_POSITION,EBIRTH,RBEGDA,RENDDA,TB_ID,urf_code_uni,SOURCE_SYSTEM,OPERATION,QUANTITY,CHANNEL_GR,IS_FRAUD,DESCRIPTION,insert_date,IN_MOTIV,PRODUCT_NAME,vl_up,DATE_UPDATE,PAYMENT_STATUS,epk_id,untb,fullname,birth,Табельный_клиента,Отпуск,ID_Operation_History,predict
4,2022-01-10,638127395.0,02424071,СКМ_ВСП,СибБ,8646,50,FSB,4202587534,783C57677CE1E58A2A159E7CBF02721F,Дебетовые карты,738.0,Momentum(личная),748.0,Личная дебетовая карта без овердрафта Сбербанк...,373.0,109.0,КАРТЫ,Не премируется,?,NaN,NaN,810,EECF0AE3AD95601F3DC9DD3F3BCAE2235F91EF19,1.0,44063226,37106329,ВСП,Индивидуальная,0,?,?,2022-01-10,2022-01-31,Присекарь Елена Васильевна,1493774.0,СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР,1997-08-18,2019-04-04,9999-12-31,44.0,044_8646_050,АС ФС,Выдача банковской карты,"1,00",ВСП,18,"Не премируются операция выдачи карт Моментум, ...",2021-11-10,0,Личная дебетовая карта без овердрафта Сбербанк...,NaN,2022-02-14,0,1.129545e+18,44063226.0,ТРУБНИКОВА АЛЕНА СЕРГЕЕВНА,1980-12-15,NaN,NaN,4,1
25,2022-01-10,416571397.0,?,ЗРО,СРБ,8639,76,ERIB,35893430305,37A6BED16B9BC5BD40082744FD132AB1,Обслуживание ФЛ,740.0,Платежи и переводы,763.0,Перевод по карте/счету,736.0,510.0,ПРОЧЕЕ,Пилот - Перевод по карте/счету,?,"100000,00",NaN,?,?,NaN,1029457695,?,?,Платеж по карте/счету,0,?,1,2022-01-10,2022-01-31,Первунина Светлана Александровна,1762560.0,ЗАМЕСТИТЕЛЬ РУКОВОДИТЕЛЯ ОФИСА,1983-06-25,2021-12-01,9999-12-31,40.0,040_8639_076,АС ЕРИБ,Перевод по карте/счету (через СБОЛ),"1,00",ВСП,0,Операция принята к учету,2021-11-10,3,Перевод по карте/счету,NaN,2022-02-14,?,1.130261e+18,41706602.0,ШИПУЛИН ВЛАДИМИР БОРИСОВИЧ,1959-06-24,NaN,NaN,25,1
26,2022-01-10,NaN,?,ЗРО,СРБ,8639,76,EPS,1323211174153,2F756D2C0DD9E696936D952E2048DA52,Страхование,327.0,Пакетные страховые продукты,405.0,"Коробочное страхование банковских карт ""Сберег...",682.0,119.0,БАНКОВСКОЕ СТРАХОВАНИЕ,"Коробочное страхование банковских карт ""Сберег...",0002.0012.0059,"1600,00","1600,00",810,?,NaN,3597025249,?,СБОЛ,Страховые коробки: Сбереги финансы (стоимость ...,1,СКМ_ВСП,0,2022-01-10,2022-01-31,Первунина Светлана Александровна,1762560.0,ЗАМЕСТИТЕЛЬ РУКОВОДИТЕЛЯ ОФИСА,1983-06-25,2021-12-01,9999-12-31,40.0,040_8639_076,АС ЕПС,Продажа продукта коробочного страхования – пер...,"1,00",ВСП,0,NaN,NaN,1,"Коробочное страхование банковских карт ""Сберег...","24,00",2022-02-14,0,1.130261e+18,41706602.0,ШИПУЛИН ВЛАДИМИР БОРИСОВИЧ,1959-06-24,NaN,NaN,26,1
36,2022-01-10,298762953.0,29621929087,СКМ_ВСП,ЮЗБ,5230,0723,COD,28468316460003,C9618C567D5754F644D2C4A1E53FF4BC,"Вклады, ОМС и инвестиционные продукты",325.0,Открытие счета,747.0,Вклады,378.0,111.0,ВКЛАДЫ И ИНВЕСТ. ПРОДУКТЫ,Открытие вклада (от 10 тыс. руб.),0002.0001.0001,"224600,00","224600,00",810,19A1574D623FB72B4CBDB5442878B0504535F7ED,NaN,177406853002,80933993202,?,Активный возраст,0,СКМ_ВСП,0,2022-01-10,2022-01-31,Анохина Ирина Ивановна,309603.0,СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР,1985-07-09,2021-08-17,9999-12-31,52.0,052_5230_0723,АС ЦОД,Открытие вклада / ОМС,"1,00",ВСП,6,Не премируется. Не первый вкалад открытый клие...,2021-11-10,1,Вклады,"24,00",2022-02-14,?,1.126925e+18,44156418.0,СМОЛЕВА ГАЛИНА ИВАНОВНА,1957-06-12,NaN,NaN,36,1
37,2022-01-10,298762953.0,29621929087,СКМ_ВСП,ЮЗБ,5230,0723,COD,28468316219103,C6196C95CDC13018015E6F71660E4DB2,"Вклады, ОМС и инвестиционные продукты",325.0,Открытие счета,747.0,Вклады,378.0,111.0,ВКЛАДЫ И ИНВЕСТ. ПРОДУКТЫ,Открытие вклада (от 10 тыс. руб.),0002.0001.0001,"500000,00","500000,00",810,84AE730B68AB1793E5E8E6228AF50B658C0D13C3,NaN,177406853002,80933992702,?,Активный возраст,1,СКМ_ВСП,0,2022-01-10,2022-01-31

In [38]:
norm_struct = motiv[motiv['predict'] == 1][['ID_Operation_History', 'DATE_REPORT', 'DATE_REPORT_SRC', 'DATE_MONTH', 'CLIENT_TID',
                     'EMPLOYEE_LOGIN', 'EMPLOYEE_FIO', 'EMPLOYEE_ID', 'EMPLOYEE_POSITION', 'employee_position_gr',
                     'TB_NAME', 'TB_ID', 'OSB_ID', 'VSP_ID', 'SOURCE_SYSTEM', 'SOURCE_SYSTEM_ID', 'ID_VDMRB',
                     'ID_VDMRB_UNIQUE', 'PRODUCT_GROUP', 'PRODUCT_GROUP_ID', 'PRODUCT_SUBGROUP', 'PRODUCT_SUBGROUP_ID',
                     'PRODUCT', 'PRODUCT_ID', 'OPERATION', 'OPERATION_ID', 'GROUP_CALC', 'PRODUCT_CALC', 'ID_CALC',
                     'QUANTITY', 'AMOUNT_RUR', 'CURRENCY', 'ACC', 'TERM', 'CLIENT_ID', 'AGREEMENT_NUM', 'CHANNEL',
                     'CHANNEL_GR', 'IS_FRAUD', 'IN_MOTIV', 'PRODUCT_SOURCE', 'PRODUCT_NAME', 'vl_up', 'PAYMENT_STATUS',
                     'IN_SUP', 'epk_id', 'prod_kpi', 'is_sbolpro', 'predict']]

In [27]:
norm_struct.dtypes

ID_Operation_History      int64
DATE_REPORT              object
DATE_REPORT_SRC          object
DATE_MONTH               object
CLIENT_TID              float64
EMPLOYEE_LOGIN           object
EMPLOYEE_FIO             object
EMPLOYEE_ID             float64
EMPLOYEE_POSITION        object
employee_position_gr     object
TB_NAME                  object
TB_ID                   float64
OSB_ID                   object
VSP_ID                   object
SOURCE_SYSTEM            object
SOURCE_SYSTEM_ID         object
ID_VDMRB                 object
ID_VDMRB_UNIQUE          object
PRODUCT_GROUP            object
PRODUCT_GROUP_ID        float64
PRODUCT_SUBGROUP         object
PRODUCT_SUBGROUP_ID     float64
PRODUCT                  object
PRODUCT_ID              float64
OPERATION                object
OPERATION_ID            float64
GROUP_CALC               object
PRODUCT_CALC             object
ID_CALC                  object
QUANTITY                 object
AMOUNT_RUR               object
CURRENCY

In [44]:
norm_struct.head(100).to_excel('../top100_norm.xlsx')

In [42]:
norm_struct['epk_id'] = norm_struct['epk_id'].astype('Int64').astype('str')

In [43]:
norm_struct

,ID_Operation_History,DATE_REPORT,DATE_REPORT_SRC,DATE_MONTH,CLIENT_TID,EMPLOYEE_LOGIN,EMPLOYEE_FIO,EMPLOYEE_ID,EMPLOYEE_POSITION,employee_position_gr,TB_NAME,TB_ID,OSB_ID,VSP_ID,SOURCE_SYSTEM,SOURCE_SYSTEM_ID,ID_VDMRB,ID_VDMRB_UNIQUE,PRODUCT_GROUP,PRODUCT_GROUP_ID,PRODUCT_SUBGROUP,PRODUCT_SUBGROUP_ID,PRODUCT,PRODUCT_ID,OPERATION,OPERATION_ID,GROUP_CALC,PRODUCT_CALC,ID_CALC,QUANTITY,AMOUNT_RUR,CURRENCY,ACC,TERM,CLIENT_ID,AGREEMENT_NUM,CHANNEL,CHANNEL_GR,IS_FRAUD,IN_MOTIV,PRODUCT_SOURCE,PRODUCT_NAME,vl_up,PAYMENT_STATUS,IN_SUP,epk_id,prod_kpi,is_sbolpro,predict
4,4,2022-01-10,2022-01-10,2022-01-31,638127395.0,02424071,Присекарь Елена Васильевна,1493774.0,СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР,СКМ_ВСП,СибБ,44.0,8646,50,АС ФС,FSB,4202587534,783C57677CE1E58A2A159E7CBF02721F,Дебетовые карты,738.0,Momentum(личная),748.0,Личная дебетовая карта без овердрафта Сбербанк...,373.0,Выдача банковской карты,109.0,КАРТЫ,Не премируется,?,"1,00",NaN,810,EECF0AE3AD95601F3DC9DD3F3BCAE2235F91EF19,1.0,44063226,37106329,ВСП,ВСП,18,0,Индивидуальная,Личная дебетовая карта без овердрафта Сбербанк...,NaN,0,0,1129545300604474112,?,?,1
25,25,2022-01-10,2022-01-10,2022-01-31,416571397.0,?,Первунина Светлана Александровна,1762560.0,ЗАМЕСТИТЕЛЬ РУКОВОДИТЕЛЯ ОФИСА,ЗРО,СРБ,40.0,8639,76,АС ЕРИБ,ERIB,35893430305,37A6BED16B9BC5BD40082744FD132AB1,Обслуживание ФЛ,740.0,Платежи и переводы,763.0,Перевод по карте/счету,736.0,Перевод по карте/счету (через СБОЛ),510.0,ПРОЧЕЕ,Пилот - Перевод по карте/счету,?,"1,00","100000,00",?,?,NaN,1029457695,?,?,ВСП,0,3,Платеж по карте/счету,Перевод по карте/счету,NaN,?,0,1130261386270937856,?,1,1
26,26,2022-01-10,2022-01-10,2022-01-31,NaN,?,Первунина Светлана Александровна,1762560.0,ЗАМЕСТИТЕЛЬ РУКОВОДИТЕЛЯ ОФИСА,ЗРО,СРБ,40.0,8639,76,АС ЕПС,EPS,1323211174153,2F756D2C0DD9E696936D952E2048DA52,Страхование,327.0,Пакетные страховые продукты,405.0,"Коробочное страхование банковских карт ""Сберег...",682.0,Продажа продукта коробочного страхования – пер...,119.0,БАНКОВСКОЕ СТРАХОВАНИЕ,"Коробочное страхование банковских карт ""Сберег...",0002.0012.0059,"1,00","1600,00",810,?,NaN,3597025249,?,СБОЛ,ВСП,0,1,Страховые коробки: Сбереги финансы (стоимость ...,"Коробочное страхование банковских карт ""Сберег...","24,00",0,1,1130261386270937856,СКМ_ВСП,0,1
36,36,2022-01-10,2022-01-10,2022-01-31,298762953.0,29621929087,Анохина Ирина Ивановна,309603.0,СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР,СКМ_ВСП,ЮЗБ,52.0,5230,0723,АС ЦОД,COD,28468316460003,C9618C567D5754F644D2C4A1E53FF4BC,"Вклады, ОМС и инвестиционные продукты",325.0,Открытие счета,747.0,Вклады,378.0,Открытие вклада / ОМС,111.0,ВКЛАДЫ И ИНВЕСТ. ПРОДУКТЫ,Открытие вклада (от 10 тыс. руб.),0002.0001.0001,"1,00","224600,00",810,19A1574D623FB72B4CBDB5442878B0504535F7ED,NaN,177406853002,80933993202,?,ВСП,6,1,Активный возраст,Вклады,"24,00",?,0,1126925386813535104,СКМ_ВСП,0,1
37,37,2022-01-10,2022-01-10,2022-01-31,298762953.0,29621929087,Анохина Ирина Ивановна,309603.0,СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР,СКМ_ВСП,ЮЗБ,52.0,5230,0723,АС ЦОД,COD,28468316219103,C6196C95CDC13018015E6F71660E4DB2,"Вклады, ОМС и инвестиционные продукты",325.0,Открытие счета,747.0,Вклады,378.0,Открытие вклада / ОМС,111.0,ВКЛАДЫ И ИНВЕСТ. ПРОДУКТЫ,Открытие вклада (от 10 тыс. руб.),0002.0001.0001,"1,00","500000,00",810,84AE730B68AB1793E5E8E6228AF50B658C0D13C3,NaN,177406853002,80933992702,?,ВСП,0,1,Активный возраст,Вклады,"24,00",?,1,1126925386813535104,СКМ_ВСП,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2750658,2750658,2022-01-29,2022-01-29,2022-01-31,NaN,?,?,NaN,?,?,Неопределе,0.0,?,?,АС ЕПС,EPS,1335551704587,?,Страхование,327.0,Продукты сложного страхования (НСЖ/ИСЖ),792.0,Накопительное страхование жизни (НСЖ) пакетное,406.0,Продажа продукта коробочного страхования – вто...,120.0,БАНКОВСКОЕ СТРАХОВАНИЕ,Не премируется,?,"1,00","2000,00",810,?,NaN,1960813275,?,АП,?,11,0,Страховые коробки: НСЖ,Накопительное 

In [45]:
norm1, norm2, norm3, norm4, norm = np.array_split(norm_struct, 4)

In [47]:
l = np.array_split(norm_struct, 10)

In [49]:
for i in tqdm(range(len(l))):
    l[i].to_excel(f'../../for_files/norm{i}.xlsx')

  0%|          | 0/10 [00:00<?, ?it/s]